In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch
import torch.nn.functional as F

from torch.optim import Adam, RMSprop
from torch.autograd import Variable
from torch.autograd import Variable
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor, Normalize
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
class FocalLoss(nn.Module):
    r"""
        This criterion is a implemenation of Focal Loss, which is proposed in 
        Focal Loss for Dense Object Detection.
            
            Loss(x, class) = - \alpha (1-softmax(x)[class])^gamma \log(softmax(x)[class])
    
        The losses are averaged across observations for each minibatch.
        Args:
            alpha(1D Tensor, Variable) : the scalar factor for this criterion
            gamma(float, double) : gamma > 0; reduces the relative loss for well-classiﬁed examples (p > .5), 
                                   putting more focus on hard, misclassiﬁed examples
            size_average(bool): size_average(bool): By default, the losses are averaged over observations for each minibatch.
                                However, if the field size_average is set to False, the losses are
                                instead summed for each minibatch.
    """
    def __init__(self, class_num, alpha=None, gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        if alpha is None:
            self.alpha = Variable(torch.ones(class_num, 1))
        else:
            if isinstance(alpha, Variable):
                self.alpha = alpha
            else:
                self.alpha = Variable(alpha)
        self.gamma = gamma
        self.class_num = class_num
        self.size_average = size_average

    def forward(self, inputs, targets):
        N = inputs.size(0)
        C = inputs.size(1)
        P = F.softmax(inputs)

        class_mask = inputs.data.new(N, C).fill_(0)
        class_mask = Variable(class_mask)
        ids = targets.view(-1, 1)
        class_mask.scatter_(1, ids.data, 1.)
        #print(class_mask)
        

        if inputs.is_cuda and not self.alpha.is_cuda:
            self.alpha = self.alpha.cuda()
        alpha = self.alpha[ids.data.view(-1)]
        
        probs = (P*class_mask).sum(1).view(-1,1)

        log_p = probs.log()

        batch_loss = -alpha*(torch.pow((1-probs), self.gamma))*log_p 
        
        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()
        return loss


In [ ]:
def initialize_weights(*models):
    for model in models:
        for module in model.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.kaiming_normal(module.weight)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.fill_(1)
                module.bias.data.zero_()
        
class AlexNet(nn.Module):

    def __init__(self, n_channel_input=1, n_channel_output=1):
        super(AlexNet, self).__init__()
        self.conv_stage_1 = nn.Sequential(
            nn.Conv2d(n_channel_input, 64, kernel_size=7, stride=1, padding=3), # 224
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 112
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 56
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 28
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 14
            nn.Conv2d(384, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 7
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512*7*7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, n_channel_output),
        )
        
        initialize_weights(self)

    def forward(self, x):
        batch_size = x.size()[0]
        x_stage_1 = self.conv_stage_1(x)
        x_stage_1 = x_stage_1.view(x.size(0), 512*7*7)
        output = self.classifier(x_stage_1)
        
        return output

In [ ]:
class Discriminator(object):
    '''
        Discriminator Network
    '''
    def __init__(self, D):
        super(Discriminator, self).__init__()
        
        # define hyper-parameters
        self.batch_size_train = 10
        self.batch_size_validation = 10
        
        # construct D
        self.D = D
        
        # define learning rate
        self.d_lr = 1e-3
        
        # define RMSprop optimizers for D and S
        self.d_optimizer = torch.optim.Adam(self.D.parameters(), lr=self.d_lr)
        
        # set training iterations
        self.epoches = 100

        # store losses
        self.D_train_loss_list = []
        self.D_validation_accuracy_list = []
        
        self.X_D_train = []
        self.X_D_validation = []
        
        # set CUDA state
        self.cuda = True
        self.D.cuda()
        
        # define loss functions
        self.CrossEntropyLoss = nn.CrossEntropyLoss().cuda()
        self.FocalLoss = FocalLoss(class_num=2).cuda()
        
    # set requies_grad=Fasle to avoid computation
    def set_requires_grad(self, nets, requires_grad=False):
        if not isinstance(nets, list):
            nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad
    
    def train(self, train_dataset, validation_dataset):
        
        for epoch in range(self.epoches):
            
            # Load dataset each epoch with SHUFFLE
            train_loader = DataLoader(train_dataset, num_workers=4, batch_size=self.batch_size_train, shuffle=True)
            
            validation_loader = DataLoader(validation_dataset, num_workers=4, batch_size=self.batch_size_validation, shuffle=True)
            iter_validation = iter(validation_loader)
            
            train_loss = 0
            validation_accuracy = 0
            
            count_train = 0
            count_validation = 0
            
            for step, (images, targets) in enumerate(train_loader):
                
                # Check for batch to have full batch_size
                if images.size()[0] != self.batch_size_train:
                    break
                    
                # Load each batch
                images = Variable(images.cuda())
                targets = Variable(targets.cuda())

                self.set_requires_grad(self.D, True)
                
                # Train D
                self.d_optimizer.zero_grad()
                
                d_predictions = self.D(images)
                # print(F.sigmoid(d_predictions))
                d_loss = self.FocalLoss(d_predictions, targets)
                d_loss.backward()
                
                # Optimize D
                self.d_optimizer.step()  
                
                '''
                batch_train_accuracy = accuracy_score(targets.cpu().data.float().numpy().tolist(), 
                                     np.argmax(F.softmax(d_predictions).cpu().data.numpy(), axis=1).tolist())
                train_accuracy = train_accuracy + batch_train_accuracy
                '''
                print("Train loss of batch %d of epoch %d: %f." % (step, epoch, d_loss.item()))
                
                train_loss = train_loss + d_loss.item()
                count_train = count_train + 1
                
                if step % 18 == 0:
                    
                    images_validation, targets_validation = iter_validation.next()

                    images_validation = Variable(images_validation.cuda())
                    targets_validation = Variable(targets_validation.cuda())

                    d_predictions_validation = self.D.forward(images_validation)

                    batch_validation_accuracy = accuracy_score(targets_validation.cpu().data.float().numpy().tolist(), 
                                         np.argmax(F.softmax(d_predictions_validation).cpu().data.numpy(), axis=1).tolist())
                    validation_accuracy = validation_accuracy + batch_validation_accuracy
                    print("Validation accuracy of batch %d of epoch %d: %f." % (count_validation, epoch, batch_validation_accuracy))
                    print("---------------------------------------------------------------")
                    
                    count_validation = count_validation + 1

            self.D_train_loss_list.append(train_loss / count_train)
            self.X_D_train.append(epoch)
            
            self.D_validation_accuracy_list.append(batch_validation_accuracy / count_validation)   
            self.X_D_validation.append(epoch)
                
            clear_output(wait=True)
            plt.plot(self.X_D_train, self.D_train_loss_list, 'r--', 
                     self.X_D_validation, self.D_validation_accuracy_list, 'g^')
            plt.show()           
            
            torch.save(self.D, './model/Alexnet_trained_model_focal_'+str(epoch)+'.pt')
            
if __name__ == '__main__':

    # Load the complete datasets for training and validation
    dataset_NEXPERIA_train = datasets.ImageFolder('./data/train/',
                                 transforms.Compose([
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 # normalize,
                             ]))
    
    dataset_NEXPERIA_validation = datasets.ImageFolder('./data/validation/',
                                 transforms.Compose([
                                 transforms.ToTensor(),
                                 # normalize,
                             ]))
        
    D = AlexNet(n_channel_input=3, n_channel_output=2)
    # D = torch.load('./model/Alexnet_trained_model_focal_0.pt')
    
    model = Discriminator(D)
    model.train(dataset_NEXPERIA_train, dataset_NEXPERIA_validation)

In [ ]:
from sklearn.metrics import accuracy_score

dataset_NEXPERIA = datasets.ImageFolder('./data/validation/',
                                 transforms.Compose([
                                 transforms.ToTensor(),
                                 # normalize,
                             ]))

train_loader = DataLoader(dataset_NEXPERIA, num_workers=4, batch_size=1, shuffle=True)

predictions_0 = []
labels_0 = []
predictions_1 = []
labels_1 = []

D = AlexNet(n_channel_input=3, n_channel_output=2)
D = torch.load('./model/Alexnet_trained_model_focal_5.pt')

for step, (images, targets) in enumerate(train_loader):

    # Load each batch
    images = Variable(images.cuda())
    targets = Variable(targets.cuda())

    d_predictions = F.softmax(D(images))

    if targets.cpu().data.numpy()[0] == 0:
        predictions_0.append(np.argmax(d_predictions.cpu().data.numpy()))
        labels_0.append(targets.cpu().data.numpy()[0])
    else:
        predictions_1.append(np.argmax(d_predictions.cpu().data.numpy()))
        labels_1.append(targets.cpu().data.numpy()[0])
    
    if step % 1000 == 0:
        print("Accuracy of first %d samples: %f for 0 samples and %f for 1 samples." % 
            (step, accuracy_score(labels_0, predictions_0), accuracy_score(labels_1, predictions_1)))
              
print("Accuracy of all 0 samples: %f." % accuracy_score(labels_0, predictions_0))
print("Accuracy of all 1 samples: %f." % accuracy_score(labels_1, predictions_1))

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

dataset_NEXPERIA = datasets.ImageFolder('./data/validation/',
                                 transforms.Compose([
                                 transforms.ToTensor(),
                                 # normalize,
                             ]))

train_loader = DataLoader(dataset_NEXPERIA, num_workers=4, batch_size=1, shuffle=True)

predictions = []
labels = []

D = AlexNet(n_channel_input=3, n_channel_output=2)
D = torch.load('./model/Alexnet_trained_model_focal_5.pt')

for step, (images, targets) in enumerate(train_loader):

    # Load each batch
    images = Variable(images.cuda())
    targets = Variable(targets.cuda())

    d_predictions = F.softmax(D(images))
    
    label = np.argmax(d_predictions.cpu().data.numpy())
    labels.append(targets.cpu().data)
    
    if label == 0:
        predictions.append(1.0 - d_predictions.cpu().data.numpy()[0,0])
    else:
        predictions.append(d_predictions.cpu().data.numpy()[0,1])
        
roc_auc_score(labels, predictions, average='weighted')

In [ ]:
import numpy as np
import os

from PIL import Image

from torch.utils.data import Dataset

from torchvision.transforms import Compose
from torchvision.transforms import ToTensor, Normalize

from scipy.io import loadmat

EXTENSIONS = ['.jpg', '.png', 'mat']

def load_image(file):
    return Image.open(file)

def is_image(filename):
    return any(filename.endswith(ext) for ext in EXTENSIONS)

def image_path(root, basename, extension):
    return os.path.join(root, basename+extension)

def image_basename(filename):
    return os.path.basename(os.path.splitext(filename)[0])

class NEXPERIA(Dataset):

    def __init__(self, root):
        self.images_root = os.path.join(root, 'image')

        self.filenames = [image_basename(f)
            for f in os.listdir(self.images_root) if is_image(f)]
        self.filenames.sort()

        self.input_transform = Compose([
            ToTensor()
        ])

    def __getitem__(self, index):
        filename = self.filenames[index]

        with open(image_path(self.images_root, filename, '.jpg'), 'rb') as f:
            image = load_image(f).convert('RGB')
        
        if self.input_transform is not None:
            image = self.input_transform(image)

        return image, filename

    def __len__(self):
        return len(self.filenames)

In [ ]:
import csv

dataset_NEXPERIA_test = NEXPERIA('./data/test')

test_loader = DataLoader(dataset_NEXPERIA_test, num_workers=4, batch_size=1, shuffle=False)

D = AlexNet(n_channel_input=3, n_channel_output=2)
D = torch.load('./model/Alexnet_trained_model_focal_5.pt')

with open('./data/submission.csv', 'w') as f:
    
    writer = csv.writer(f)
    
    header = ['id', 'label']
    writer.writerow(header)
    
    for step, (image, filename) in enumerate(test_loader):

        # Load each batch
        image = Variable(image.cuda())

        d_predictions = F.softmax(D(image))

        label = np.argmax(d_predictions.cpu().data.numpy())

        if label == 0:
            prob = 1.0 - d_predictions.cpu().data.numpy()[0,0]
        else:
            prob = d_predictions.cpu().data.numpy()[0,1]
        
        row = [filename[0], float(prob)]
        writer.writerow(row)
        
        print(filename[0])
        
f.close()
    